In [2]:
import boto3
import numpy
import pandas

import constants
from solver import enrich_data_for_train
from utils.io.s3 import download_dataframe, upload_dataframe

In [3]:
s3_session = boto3.session.Session()
s3_client = s3_session.client(service_name='s3', endpoint_url='https://storage.yandexcloud.net')

In [4]:
dataset_df = download_dataframe(s3_client, constants.S3_BUCKET, constants.DATA_PATH / 'train.parquet')
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2872562 entries, 0 to 2872561
Data columns (total 2 columns):
 #   Column      Dtype
---  ------      -----
 0   uid         int64
 1   friend_uid  int64
dtypes: int64(2)
memory usage: 43.8 MB


In [5]:
dataset_df = dataset_df.sample(frac=0.01, random_state=94)

In [6]:
dataset_df.describe()

,uid,friend_uid
count,28726.000000,28726.000000
mean,38251.947574,79131.462403
std,27179.773768,28380.548545
min,0.000000,718.000000
25%,15253.750000,58743.000000
50%,33598.500000,83791.000000
75%,57176.000000,103094.750000
max,117356.000000,120060.000000


In [7]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import ToUndirected

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [9]:
edges = torch.from_numpy(dataset_df.to_numpy())

data = Data(edge_index=edges.T.contiguous())
data = ToUndirected()(data)
data.validate()
data

/opt/conda/envs/recsys/lib/python3.8/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


Data(edge_index=[2, 57452])

In [12]:
data.num_nodes

/opt/conda/envs/recsys/lib/python3.8/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


120061

In [10]:
model = Node2Vec(
    data.edge_index,
    embedding_dim=32,  # размер эмбеддинга вершины
    walk_length=20,  # длина случайного блуждания
    context_size=10,  # размер окна из случайного блуждания (как в w2v)
    walks_per_node=10,  # количество случайных блужданий из одной вершины
    num_negative_samples=1,  # количество негативных примеров на один позитивный
    p=1.0,  # параметр вероятности вернуться в предыдущую вершину
    q=1.0,  # параметр вероятности исследовать граф вглубь
    sparse=True,
).to(device)

# класс Node2Vec предоставляет сразу генератор случайного блуждания
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.05)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        # pos_rw – последовательность из случайного блуждания
        # neg_rw – случайные негативные примеры
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

for epoch in range(1, 5):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 1.1725
Epoch: 002, Loss: 1.0550
Epoch: 003, Loss: 1.1116
Epoch: 004, Loss: 1.0879


In [11]:
model.embedding.weight.detach().numpy().shape

(120061, 32)